### Компиляция и линковка.

<br />

##### Этапы сборки программ

https://en.cppreference.com/w/cpp/language/translation_phases

Упрощённая схема:

![](program_stages_2.jpg)

Вопросы:
* где в этом процессе header-файлы?
* как собирать не исполняемый файл, а статическую билиотеку?
* как в этой схеме обрабатываются шаблоны? (вопрос с двойным подвохом)
* на каких уровнях возможна параллелизация процесса?
* как сюда вписываются precompiled headers?

<br />

##### Препроцессор - зачем и как

Заголовочные файлы

```c++
#include <string>

#include "utils.h"
```

Макросы автоподстановки - постарайтесь по возможности избегать их, предпочитать С++ - аналоги

```c++
#define SQR(x)  x * x
```

<details>
<summary>Объяснение ошибки</summary>
<p>

```c++
// error:
//     SQR(2 + 3)  --> 2 + 3 * 2 + 3 == 11  // OOOOOPS!
//
// corrected version:
#define SQR(x) (x)*(x)
```

</p>
</details>

<br />
Другой пример:

```c++
#define LOG(msg) \
    if (is_logging_enabled) \
        std::clog << (msg) << std::endl;
```

<details>
<summary>Объяснение ошибки</summary>
<p>

```c++
// error: 
//     if (something_bad_happened)
//         LOG("something bad happened!");
//     else
//         run_task();  // OOOOPS!
//
// corrected version (linux kernel programming guides):
#define LOG(msg) \
    do { \
        if (is_logging_enabled) \
            std::clog << (msg) << std::endl; \
    } while(0)
```

</p>
</details>

Генерация ошибок и предупреждений в compile-time

```c++
#error This file must not be included into compilation!
```

```c++
void f()
{
    #warning Function f() must be fixed for case when there is no network connection
    ...
}
```

Условная компиляция

```c++
// условная компиляция под разные ОС
void remove_file(const std::string& filename)
{
    #if defined(__LINUX__)
        ...
    #elif defined(__WINDOWS__)
        ...
    #elif defined(__MACOSX__)
        ...
    #elif defined(__ANDROID__)
        ...
    #elif defined(__IOS__)
        ...
    #else
        #error Unsupported target OS
    #endif
}

// условная компиляция для переключения фичей через пользовательские флаги
// (например, -DENABLE_LOGGING для gcc/clang)
#ifdef ENABLE_LOGGING
    #define LOG(msg) \
            std::clog << (msg) << std::endl;
#else
    #define LOG(msg) ;
#endif

// условная компиляция на разных компиляторах для поддержки расширений:
#if defined(__GNUC__) || defined(__clang__)
    #define ALWAYS_INLINE __attribute__((noinline))
#elif defined(_MSC_VER)
    #define ALWAYS_INLINE __declspec(noinline)
#else
    #define ALWAYS_INLINE
    #warning ALWAYS_INLINE macro is not supported for this compiler
#endif

ALWAYS_INLINE void f() {
    ...
}

ALWAYS_INLINE int g() {
    ...
}
```

https://en.cppreference.com/w/cpp/preprocessor/replace

https://en.cppreference.com/w/cpp/preprocessor/impl

А также:
* информация о поддерживаемых фичах компилятором (`__cpp_generic_lambdas`, `__cpp_range_based_for` и др.)
* дополнительная информация для точной отладки/трассировки/профилировки (`__LINE__`, `__FILE__` и нестандартный `__function__`)
* `#pragma` - команды либо подсказки компилятору (`#pragma once`, `#pragma unroll` ...)

<br />

##### Компиляция. Содержимое объектных файлов. Область видимости символов.

Стадии компиляции (упрощённо):
* Лексический анализ. Разбивает код программы на последовательность лексем.
* Синтаксический анализ. Определение порядка выполнения команд, построение AST/DAG из лексем.
* Семантический анализ. Применение семантических правил языка к AST/DAG.
* Оптимизация и кодогенерация. На выходе - объектный файл.

__Вопрос-наброс__: какая из стадий самая дорогая по времени?

<br />

Давайте набросаем простенькую программку из нескольких файлов и посмотрим на содержимое объектных файлов

In [ ]:
# %load ex1_objfiles/main.cpp
#include "math_utils.h"
#include "string_utils.h"

#include <cstdio>
#include <cstring>

void print_help() noexcept
{
    std::puts("USAGE:");
    std::puts("    exmaple.bin msum 3 4");
    std::puts("        7");
    std::puts("    exmaple.bin mmul 3 4");
    std::puts("        12");
    std::puts("    exmaple.bin ssum 3 4");
    std::puts("        34");
    std::puts("    exmaple.bin smul 3 4");
    std::puts("        3333");
    std::puts("");
    std::puts("Errors processing is not implemented in order to simplify example");
}

int main(int argc, char** argv)
{
    if (argc != 4)
    {
        print_help();
        return 1;
    }

    const char* const op = argv[1];
    const char* const a1 = argv[2];
    const char* const a2 = argv[3];
    if (!std::strcmp(op, "msum"))
        math_utils::sum(a1, a2);
    else if (!std::strcmp(op, "mmul"))
        math_utils::mul(a1, a2);
    else if (!std::strcmp(op, "ssum"))
        string_utils::sum(a1, a2);
    else if (!std::strcmp(op, "smul"))
        string_utils::mul(a1, a2);
    return 0;
}

In [ ]:
# %load ex1_objfiles/math_utils.h
#pragma once

namespace math_utils
{
    void sum(const char* a1, const char* a2) noexcept;
    void mul(const char* a1, const char* a2) noexcept;
}  // namespace math_utils

In [ ]:
# %load ex1_objfiles/string_utils.h
#pragma once

namespace string_utils
{
    void sum(const char* a1, const char* a2) noexcept;
    void mul(const char* a1, const char* a2) noexcept;
}  // namespace string_utils

In [ ]:
# %load ex1_objfiles/math_utils.cpp
#include "math_utils.h"

#include <cstdio>
#include <cstdlib>

int convert_to_int(const char* const s) noexcept
{
    return std::atoi(s);
}

namespace math_utils
{
    void sum(const char* const a1, const char* const a2) noexcept
    {
        const int x1 = convert_to_int(a1);
        const int x2 = convert_to_int(a2);
        std::printf("%d\n", x1 + x2);
    }

    void mul(const char* const a1, const char* const a2) noexcept
    {
        const int x1 = convert_to_int(a1);
        const int x2 = convert_to_int(a2);
        std::printf("%d\n", x1 * x2);
    }
}  // namespace math_utils

In [ ]:
# %load ex1_objfiles/string_utils.cpp
#include "string_utils.h"

#include <cstdio>
#include <cstdlib>

int convert_to_int(const char* const s) noexcept
{
    return std::atoi(s);
}

namespace string_utils
{
    void sum(const char* const a1, const char* const a2) noexcept
    {
        std::puts(a1);
        std::puts(a2);
    }

    void mul(const char* const a1, const char* const a2) noexcept
    {
        const int x2 = convert_to_int(a2);
        for (int i = 0; i < x2; ++i)
            std::puts(a1);
    }
}  // namespace string_utils

Скомпилируем объектные файлы:

In [16]:
!clang++ -c -Os ex1_objfiles/main.cpp
!clang++ -c -Os ex1_objfiles/math_utils.cpp
!clang++ -c -Os ex1_objfiles/string_utils.cpp

In [17]:
!ls *.o -lh

-rw-rw-r-- 1 ivafanas ivafanas 2,7K сен  5 00:51 main.o
-rw-rw-r-- 1 ivafanas ivafanas 1,7K сен  5 00:51 math_utils.o
-rw-rw-r-- 1 ivafanas ivafanas 1,4K сен  5 00:51 string_utils.o


Посмотрим на символы в main.o:

In [18]:
!objdump -t main.o


main.o:     file format elf64-x86-64

SYMBOL TABLE:
0000000000000000 l    df *ABS*	0000000000000000 main.cpp
0000000000000000 l    d  .text	0000000000000000 .text
0000000000000000 l    d  .rodata.str1.1	0000000000000000 .rodata.str1.1
0000000000000000 g     F .text	0000000000000070 _Z10print_helpv
0000000000000000         *UND*	0000000000000000 _ZN10math_utils3mulEPKcS1_
0000000000000000         *UND*	0000000000000000 _ZN10math_utils3sumEPKcS1_
0000000000000000         *UND*	0000000000000000 _ZN12string_utils3mulEPKcS1_
0000000000000000         *UND*	0000000000000000 _ZN12string_utils3sumEPKcS1_
0000000000000070 g     F .text	00000000000000a4 main
0000000000000000         *UND*	0000000000000000 putchar
0000000000000000         *UND*	0000000000000000 puts
0000000000000000         *UND*	0000000000000000 strcmp




Таблица символов показывает, что:
* в этом файле определены и видны наружу `print_help` и `main`
* файл ссылается на символы `math_utils::mul`, `math_utils::sum`, `string_utils::mul`, `string_utils::sum`, `putchar`, `puts`, `strcmp`, которые реализованы где-то извне

https://en.cppreference.com/w/cpp/language/storage_duration (раздел Linkage)

А вот, кстати, и наши строчки:

In [28]:
!objdump -s --section=.rodata.str1.1 main.o


main.o:     file format elf64-x86-64

Contents of section .rodata.str1.1:
 0000 55534147 453a0020 20202065 786d6170  USAGE:.    exmap
 0010 6c652e62 696e206d 73756d20 33203400  le.bin msum 3 4.
 0020 20202020 20202020 37002020 20206578          7.    ex
 0030 6d61706c 652e6269 6e206d6d 756c2033  maple.bin mmul 3
 0040 20340020 20202020 20202031 32002020   4.        12.  
 0050 20206578 6d61706c 652e6269 6e207373    exmaple.bin ss
 0060 756d2033 20340020 20202020 20202033  um 3 4.        3
 0070 34002020 20206578 6d61706c 652e6269  4.    exmaple.bi
 0080 6e20736d 756c2033 20340020 20202020  n smul 3 4.     
 0090 20202033 33333300 4572726f 72732070     3333.Errors p
 00a0 726f6365 7373696e 67206973 206e6f74  rocessing is not
 00b0 20696d70 6c656d65 6e746564 20696e20   implemented in 
 00c0 6f726465 7220746f 2073696d 706c6966  order to simplif
 00d0 79206578 616d706c 65006d73 756d006d  y example.msum.m
 00e0 6d756c00 7373756d 00736d75 6c00      mul.ssum.smul.  


Посмотрим на символы `math_utils`:

In [19]:
!objdump -t math_utils.o


math_utils.o:     file format elf64-x86-64

SYMBOL TABLE:
0000000000000000 l    df *ABS*	0000000000000000 math_utils.cpp
0000000000000000 l    d  .text	0000000000000000 .text
0000000000000000 l    d  .rodata.str1.1	0000000000000000 .rodata.str1.1
0000000000000000 g     F .text	0000000000000005 _Z14convert_to_intPKc
0000000000000033 g     F .text	000000000000002e _ZN10math_utils3mulEPKcS1_
0000000000000005 g     F .text	000000000000002e _ZN10math_utils3sumEPKcS1_
0000000000000000         *UND*	0000000000000000 atoi
0000000000000000         *UND*	0000000000000000 printf




Посмотрим на символы `string_utils`:

In [20]:
!objdump -t string_utils.o


string_utils.o:     file format elf64-x86-64

SYMBOL TABLE:
0000000000000000 l    df *ABS*	0000000000000000 string_utils.cpp
0000000000000000 l    d  .text	0000000000000000 .text
0000000000000000 g     F .text	0000000000000005 _Z14convert_to_intPKc
0000000000000017 g     F .text	0000000000000027 _ZN12string_utils3mulEPKcS1_
0000000000000005 g     F .text	0000000000000012 _ZN12string_utils3sumEPKcS1_
0000000000000000         *UND*	0000000000000000 atoi
0000000000000000         *UND*	0000000000000000 puts




__Вопросы__:
* Программа не слинкуется. Почему? Как починить?
* Почему компилятору бы не перенести `print_help` и `convert_to_int` из видимых наружу символов в локальные, ведь они никем не используются?

И немного ассемблера что творится в этих файлах:

In [21]:
!objdump -d math_utils.o


math_utils.o:     file format elf64-x86-64


Disassembly of section .text:

0000000000000000 <_Z14convert_to_intPKc>:
   0:	e9 00 00 00 00       	jmpq   5 <_ZN10math_utils3sumEPKcS1_>

0000000000000005 <_ZN10math_utils3sumEPKcS1_>:
   5:	41 56                	push   %r14
   7:	53                   	push   %rbx
   8:	50                   	push   %rax
   9:	48 89 f3             	mov    %rsi,%rbx
   c:	e8 00 00 00 00       	callq  11 <_ZN10math_utils3sumEPKcS1_+0xc>
  11:	41 89 c6             	mov    %eax,%r14d
  14:	48 89 df             	mov    %rbx,%rdi
  17:	e8 00 00 00 00       	callq  1c <_ZN10math_utils3sumEPKcS1_+0x17>
  1c:	42 8d 34 30          	lea    (%rax,%r14,1),%esi
  20:	bf 00 00 00 00       	mov    $0x0,%edi
  25:	31 c0                	xor    %eax,%eax
  27:	48 83 c4 08          	add    $0x8,%rsp
  2b:	5b                   	pop    %rbx
  2c:	41 5e                	pop    %r14
  2e:	e9 00 00 00 00       	jmpq   33 <_ZN10math_utils3mulEPKcS1_>

0000000000000033 <_ZN10math_uti

In [22]:
!objdump -d string_utils.o


string_utils.o:     file format elf64-x86-64


Disassembly of section .text:

0000000000000000 <_Z14convert_to_intPKc>:
   0:	e9 00 00 00 00       	jmpq   5 <_ZN12string_utils3sumEPKcS1_>

0000000000000005 <_ZN12string_utils3sumEPKcS1_>:
   5:	53                   	push   %rbx
   6:	48 89 f3             	mov    %rsi,%rbx
   9:	e8 00 00 00 00       	callq  e <_ZN12string_utils3sumEPKcS1_+0x9>
   e:	48 89 df             	mov    %rbx,%rdi
  11:	5b                   	pop    %rbx
  12:	e9 00 00 00 00       	jmpq   17 <_ZN12string_utils3mulEPKcS1_>

0000000000000017 <_ZN12string_utils3mulEPKcS1_>:
  17:	55                   	push   %rbp
  18:	53                   	push   %rbx
  19:	50                   	push   %rax
  1a:	48 89 fb             	mov    %rdi,%rbx
  1d:	48 89 f7             	mov    %rsi,%rdi
  20:	e8 00 00 00 00       	callq  25 <_ZN12string_utils3mulEPKcS1_+0xe>
  25:	89 c5                	mov    %eax,%ebp
  27:	85 ed                	test   %ebp,%ebp
  29:	7e 0c                

Подчистим за собой:

In [ ]:
rm -f *.o

<br />

Теперь подправим math_utils.cpp, скажем компилятору, что функция `covnert_to_int` снаружи никому не нужна:

In [ ]:
# %load ex1_objfiles/math_utils_w_static.cpp
#include "math_utils.h"

#include <cstdio>
#include <cstdlib>

static int convert_to_int(const char* const s) noexcept
{
    return std::atoi(s);
}

namespace math_utils
{
    void sum(const char* const a1, const char* const a2) noexcept
    {
        const int x1 = convert_to_int(a1);
        const int x2 = convert_to_int(a2);
        std::printf("%d\n", x1 + x2);
    }

    void mul(const char* const a1, const char* const a2) noexcept
    {
        const int x1 = convert_to_int(a1);
        const int x2 = convert_to_int(a2);
        std::printf("%d\n", x1 * x2);
    }
}  // namespace math_utils

In [37]:
!clang++ -Os -c ex1_objfiles/math_utils_w_static.cpp

In [38]:
!objdump -t math_utils_w_static.o


math_utils_w_static.o:     file format elf64-x86-64

SYMBOL TABLE:
0000000000000000 l    df *ABS*	0000000000000000 math_utils_w_static.cpp
0000000000000000 l    d  .text	0000000000000000 .text
0000000000000000 l    d  .rodata.str1.1	0000000000000000 .rodata.str1.1
000000000000002e g     F .text	000000000000002e _ZN10math_utils3mulEPKcS1_
0000000000000000 g     F .text	000000000000002e _ZN10math_utils3sumEPKcS1_
0000000000000000         *UND*	0000000000000000 atoi
0000000000000000         *UND*	0000000000000000 printf




In [39]:
!objdump -d math_utils_w_static.o


math_utils_w_static.o:     file format elf64-x86-64


Disassembly of section .text:

0000000000000000 <_ZN10math_utils3sumEPKcS1_>:
   0:	41 56                	push   %r14
   2:	53                   	push   %rbx
   3:	50                   	push   %rax
   4:	48 89 f3             	mov    %rsi,%rbx
   7:	e8 00 00 00 00       	callq  c <_ZN10math_utils3sumEPKcS1_+0xc>
   c:	41 89 c6             	mov    %eax,%r14d
   f:	48 89 df             	mov    %rbx,%rdi
  12:	e8 00 00 00 00       	callq  17 <_ZN10math_utils3sumEPKcS1_+0x17>
  17:	42 8d 34 30          	lea    (%rax,%r14,1),%esi
  1b:	bf 00 00 00 00       	mov    $0x0,%edi
  20:	31 c0                	xor    %eax,%eax
  22:	48 83 c4 08          	add    $0x8,%rsp
  26:	5b                   	pop    %rbx
  27:	41 5e                	pop    %r14
  29:	e9 00 00 00 00       	jmpq   2e <_ZN10math_utils3mulEPKcS1_>

000000000000002e <_ZN10math_utils3mulEPKcS1_>:
  2e:	55                   	push   %rbp
  2f:	53                   	push   %rbx
  30:

В данном случае компилятор не просто перенёс символ `convert_to_int` в локальные (можно это продемонстрировать, убрав оптимизации), но и принял решение полностью удалить функцию, заинлайнив её.

Аналогичным образом желательно спрятать функцию `convert_to_int` в `string_utils.cpp` и `print_help` в `main.cpp`.

__Вопрос__: Что делать, если хотим избавиться от копипасты `convert_to_int`? В отдельный `cpp`? В хедер? Как себя поведёт компилятор?

<br />

##### static vs inline vs compiler inline

Попробуем избавиться от copy-paste с `convert_to_int` разными вариантами.

__Вариант 1__: вынесем `convert_to_int` в header и объявим его `static`

`static` перед функцией означает, что символ не нужно экспортировать наружу, он должен быть локальным.

In [ ]:
# %load ex2_objfiles/convert.h
#pragma once

#include <cstdlib>

static int convert_to_int(const char* const s) noexcept
{
    return std::atoi(s);
}

In [ ]:
# %load ex2_objfiles/math_utils.cpp
#include "math_utils.h"

#include "convert.h"

#include <cstdio>

namespace math_utils
{
    void sum(const char* const a1, const char* const a2) noexcept
    {
        const int x1 = convert_to_int(a1);
        const int x2 = convert_to_int(a2);
        std::printf("%d\n", x1 + x2);
    }

    void mul(const char* const a1, const char* const a2) noexcept
    {
        const int x1 = convert_to_int(a1);
        const int x2 = convert_to_int(a2);
        std::printf("%d\n", x1 * x2);
    }
}  // namespace math_utils

In [ ]:
# %load ex2_objfiles/string_utils.cpp
#include "string_utils.h"

#include "convert.h"

#include <cstdio>

namespace string_utils
{
    void sum(const char* const a1, const char* const a2) noexcept
    {
        std::puts(a1);
        std::puts(a2);
    }

    void mul(const char* const a1, const char* const a2) noexcept
    {
        const int x2 = convert_to_int(a2);
        for (int i = 0; i < x2; ++i)
            std::puts(a1);
    }
}  // namespace string_utils

Скомпилируем, обратим особое внимание на символ `convert_to_int`

In [49]:
!clang++ -O0 -c ex2_objfiles/main.cpp
!clang++ -O0 -c ex2_objfiles/math_utils.cpp
!clang++ -O0 -c ex2_objfiles/string_utils.cpp
!clang++ -O0 main.o math_utils.o string_utils.o -o a.out

In [64]:
!objdump -t math_utils.o | grep convert_to_int

0000000000000060 l     F .text	000000000000001b _ZL14convert_to_intPKc


In [66]:
!objdump -t string_utils.o | grep convert_to_int

00000000000000c0 l     F .text	000000000000001b _ZL14convert_to_intPKc


In [65]:
!objdump -t a.out | grep convert_to_int

00000000004009b0 l     F .text	000000000000001b              _ZL14convert_to_intPKc
0000000000400af0 l     F .text	000000000000001b              _ZL14convert_to_intPKc


То же самое с оптимизациями:

In [67]:
!clang++ -Os -c ex2_objfiles/main.cpp
!clang++ -Os -c ex2_objfiles/math_utils.cpp
!clang++ -Os -c ex2_objfiles/string_utils.cpp
!clang++ -Os main.o math_utils.o string_utils.o -o a.out
!objdump -t a.out | grep convert_to_int

__Итого__: 
* в случае без оптимизаций происходит то, что и должно быть: каждый объектный файл содержит свою собственную реализацию `convert_to_int`, и **функция в исполняемом файле встречается столько раз, сколько объектных файлов слинковано**
* в случае оптимизаций компилятор здесь принимает решение выкинуть функцию, заинлайнив её код. Но такое поведение зависит от размера функции и работает не всегда.

Подчистим за собой:

In [68]:
rm -f *.o a.out

<br />

__Вариант 2:__ вынесем `convert_to_int` в header и объявим его `inline`

`inline` перед функцией имеет два значения:
* позволяет нарушать ODR
* рекомендация компилятору заинлайнить код функции (показать как это работает в godbolt на clang)

Отступление: показать сообщения clang об оптимизациях на примере (потом добавить std::puts в sqr2, потом убрать static-и):

```c++
static double sqr1(double x) {
    return x * x;
}

static inline double sqr2(double x) {
    return x * x;
}

double f(double x, double y) {
    return sqr1(x) + sqr2(y);
}
```

Вернёмся к `inline` и ODR

In [ ]:
# %load ex3_objfiles/convert.h
#pragma once

#include <cstdlib>

inline int convert_to_int(const char* const s) noexcept
{
    return std::atoi(s);
}

Скомпилируем программу без оптимизаций:

In [71]:
!clang++ -O0 -c ex3_objfiles/main.cpp
!clang++ -O0 -c ex3_objfiles/math_utils.cpp
!clang++ -O0 -c ex3_objfiles/string_utils.cpp
!clang++ main.o math_utils.o string_utils.o

In [73]:
!objdump -t math_utils.o | grep convert_to_int

0000000000000000 l    d  .text._Z14convert_to_intPKc	0000000000000000 .text._Z14convert_to_intPKc
0000000000000000  w    F .text._Z14convert_to_intPKc	000000000000001b _Z14convert_to_intPKc


In [74]:
!objdump -t string_utils.o | grep convert_to_int

0000000000000000 l    d  .text._Z14convert_to_intPKc	0000000000000000 .text._Z14convert_to_intPKc
0000000000000000  w    F .text._Z14convert_to_intPKc	000000000000001b _Z14convert_to_intPKc


In [72]:
!objdump -t a.out | grep convert_to_int

0000000000400a10  w    F .text	000000000000001b              _Z14convert_to_intPKc


То же самое с опитизациями:

In [75]:
!clang++ -Os -c ex3_objfiles/main.cpp
!clang++ -Os -c ex3_objfiles/math_utils.cpp
!clang++ -Os -c ex3_objfiles/string_utils.cpp
!clang++ main.o math_utils.o string_utils.o

In [77]:
!objdump -t math_utils.o | grep convert_to_int

In [76]:
!objdump -t a.out | grep convert_to_int

__Итого:__
* в случае без оптимизаций происходит то, что и должно быть: каждый объектный файл может содержать свою собственную реализацию `convert_to_int`, а функция в исполняемом файле встречается один раз.
* в случае оптимизаций компилятор здесь принимает решение выкинуть функцию, заинлайнив её код. Но такое поведение зависит от размера функции и работает не всегда.

Подчистим за собой:

In [78]:
rm -f *.o a.out

<br />

##### Классы и анонимный namespace. Местонахождение методов класса.

Пару слов на примерах как компилятор работает с символами на примере классов

__Вариант 1__: класс с методами, определёнными в рамках класса

In [ ]:
# %load ex4_class/example_class_1.cpp
#include <cstdio>

class HTMLPrinter {
public:
    HTMLPrinter() = default;
    ~HTMLPrinter() = default;

    void p() {
        print("<p>");
    }

    void div() {
        print("<div>");
    }

private:
    void print(const char* s) {
        std::puts(s);
    }
};

void f() {
    HTMLPrinter printer;
    printer.p();
}

In [91]:
!clang++ -O0 -c ex4_class/example_class_1.cpp
!objdump -t example_class_1.o


example_class_1.o:     file format elf64-x86-64

SYMBOL TABLE:
0000000000000000 l    df *ABS*	0000000000000000 example_class_1.cpp
0000000000000000 l    d  .text	0000000000000000 .text
0000000000000000 l    d  .text._ZN11HTMLPrinter1pEv	0000000000000000 .text._ZN11HTMLPrinter1pEv
0000000000000000 l    d  .text._ZN11HTMLPrinter5printEPKc	0000000000000000 .text._ZN11HTMLPrinter5printEPKc
0000000000000000 l    d  .rodata.str1.1	0000000000000000 .rodata.str1.1
0000000000000000 g     F .text	0000000000000017 _Z1fv
0000000000000000  w    F .text._ZN11HTMLPrinter1pEv	0000000000000025 _ZN11HTMLPrinter1pEv
0000000000000000  w    F .text._ZN11HTMLPrinter5printEPKc	0000000000000022 _ZN11HTMLPrinter5printEPKc
0000000000000000         *UND*	0000000000000000 puts




In [ ]:
# %load ex4_class/example_class_2.cpp
#include <cstdio>

class HTMLPrinter {
public:
    HTMLPrinter();
    ~HTMLPrinter();

    void p();
    void div();

private:
    void print(const char* s);
};

HTMLPrinter::HTMLPrinter() = default;
HTMLPrinter::~HTMLPrinter() = default;

void HTMLPrinter::p() {
    print("<p>");
}

void HTMLPrinter::div() {
    print("<div>");
}

void HTMLPrinter::print(const char* const s) {
    std::puts(s);
}

void f() {
    HTMLPrinter printer;
    printer.p();
}

In [93]:
!clang++ -O0 -c ex4_class/example_class_2.cpp
!objdump -t example_class_2.o


example_class_2.o:     file format elf64-x86-64

SYMBOL TABLE:
0000000000000000 l    df *ABS*	0000000000000000 example_class_2.cpp
0000000000000000 l       .gcc_except_table	0000000000000000 GCC_except_table5
0000000000000000 l    d  .text	0000000000000000 .text
0000000000000000 l    d  .gcc_except_table	0000000000000000 .gcc_except_table
0000000000000000 l    d  .rodata.str1.1	0000000000000000 .rodata.str1.1
0000000000000000         *UND*	0000000000000000 _Unwind_Resume
00000000000000b0 g     F .text	0000000000000050 _Z1fv
0000000000000020 g     F .text	0000000000000025 _ZN11HTMLPrinter1pEv
0000000000000080 g     F .text	0000000000000025 _ZN11HTMLPrinter3divEv
0000000000000050 g     F .text	0000000000000022 _ZN11HTMLPrinter5printEPKc
0000000000000000 g     F .text	000000000000000a _ZN11HTMLPrinterC1Ev
0000000000000000 g     F .text	000000000000000a _ZN11HTMLPrinterC2Ev
0000000000000010 g     F .text	000000000000000a _ZN11HTMLPrinterD1Ev
0000000000000010 g     F .text	000000000000000a

Объяснить разницу

Способ сделать класс с локальным связыванием:

In [ ]:
# %load ex4_class/example_class_3.cpp
#include <cstdio>

namespace {
  class HTMLPrinter {
  public:
      HTMLPrinter();
      ~HTMLPrinter();

      void p();
      void div();

  private:
      void print(const char* s);
  };

  HTMLPrinter::HTMLPrinter() = default;
  HTMLPrinter::~HTMLPrinter() = default;

  void HTMLPrinter::p() {
      print("<p>");
  }

  void HTMLPrinter::div() {
      print("<div>");
  }

  void HTMLPrinter::print(const char* const s) {
      std::puts(s);
  }
}  // namespace

void f() {
    HTMLPrinter printer;
    printer.p();
}

In [95]:
!clang++ -O0 -c ex4_class/example_class_3.cpp
!objdump -t example_class_3.o


example_class_3.o:     file format elf64-x86-64

SYMBOL TABLE:
0000000000000000 l    df *ABS*	0000000000000000 example_class_3.cpp
0000000000000000 l       .gcc_except_table	0000000000000000 GCC_except_table0
0000000000000060 l     F .text	0000000000000025 _ZN12_GLOBAL__N_111HTMLPrinter1pEv
00000000000000a0 l     F .text	0000000000000022 _ZN12_GLOBAL__N_111HTMLPrinter5printEPKc
0000000000000050 l     F .text	000000000000000a _ZN12_GLOBAL__N_111HTMLPrinterC2Ev
0000000000000090 l     F .text	000000000000000a _ZN12_GLOBAL__N_111HTMLPrinterD2Ev
0000000000000000 l    d  .text	0000000000000000 .text
0000000000000000 l    d  .gcc_except_table	0000000000000000 .gcc_except_table
0000000000000000 l    d  .rodata.str1.1	0000000000000000 .rodata.str1.1
0000000000000000         *UND*	0000000000000000 _Unwind_Resume
0000000000000000 g     F .text	0000000000000050 _Z1fv
0000000000000000         *UND*	0000000000000000 __gxx_personality_v0
0000000000000000         *UND*	0000000000000000 puts




Подчистим за собой:

In [96]:
!rm -f *.o

<br />

##### Шаблоны и символы

Рассмотрим пример с `std::vector`

In [ ]:
# %load ex5_templates/util.h
#pragma once

#include <vector>

int sum(const std::vector<int>& v);

In [ ]:
# %load ex5_templates/util.cpp
#include "util.h"

int sum(const std::vector<int>& v)
{
    auto v_copy = v;
    int rv = 0;
    for (int x : v_copy)
        rv += x;
    return rv;
}

In [ ]:
# %load ex5_templates/main.cpp
#include "util.h"

#include <cstdio>
#include <vector>

int mul(const std::vector<int>& v)
{
    auto v_copy = v;
    int rv = 1;
    for (int x : v_copy)
        rv *= x;
    return rv;
}

int main(int argc, char** argv)
{
    std::vector<int> v = {1, 2, 3};
    std::printf("%d", mul(v));
    std::printf("%d", sum(v));
    return 0;
}

Скомпилируем, обратим внимание на `vector`

In [101]:
!clang++ -Os -c ex5_templates/main.cpp
!clang++ -Os -c ex5_templates/util.cpp
!clang++ -Os main.o util.o

In [110]:
!objdump -t util.o | grep -i vector

0000000000000000 l    d  .text._ZNSt6vectorIiSaIiEEC2ERKS1_	0000000000000000 .text._ZNSt6vectorIiSaIiEEC2ERKS1_
0000000000000000 l    d  .text._ZNSt12_Vector_baseIiSaIiEE11_M_allocateEm	0000000000000000 .text._ZNSt12_Vector_baseIiSaIiEE11_M_allocateEm
0000000000000000 g     F .text	0000000000000046 _Z3sumRKSt6vectorIiSaIiEE
0000000000000000  w    F .text._ZNSt12_Vector_baseIiSaIiEE11_M_allocateEm	0000000000000026 _ZNSt12_Vector_baseIiSaIiEE11_M_allocateEm
0000000000000000  w    F .text._ZNSt6vectorIiSaIiEEC2ERKS1_	0000000000000079 _ZNSt6vectorIiSaIiEEC2ERKS1_


In [108]:
!objdump -t main.o | grep -i vector

0000000000000000 l    d  .text._ZNSt6vectorIiSaIiEEC2ERKS1_	0000000000000000 .text._ZNSt6vectorIiSaIiEEC2ERKS1_
0000000000000000 l    d  .text._ZNSt6vectorIiSaIiEEC2ESt16initializer_listIiERKS0_	0000000000000000 .text._ZNSt6vectorIiSaIiEEC2ESt16initializer_listIiERKS0_
0000000000000000 l    d  .text._ZNSt12_Vector_baseIiSaIiEE11_M_allocateEm	0000000000000000 .text._ZNSt12_Vector_baseIiSaIiEE11_M_allocateEm
0000000000000000 g     F .text	000000000000004a _Z3mulRKSt6vectorIiSaIiEE
0000000000000000         *UND*	0000000000000000 _Z3sumRKSt6vectorIiSaIiEE
0000000000000000  w    F .text._ZNSt12_Vector_baseIiSaIiEE11_M_allocateEm	0000000000000026 _ZNSt12_Vector_baseIiSaIiEE11_M_allocateEm
0000000000000000  w    F .text._ZNSt6vectorIiSaIiEEC2ERKS1_	0000000000000079 _ZNSt6vectorIiSaIiEEC2ERKS1_
0000000000000000  w    F .text._ZNSt6vectorIiSaIiEEC2ESt16initializer_listIiERKS0_	000000000000007f _ZNSt6vectorIiSaIiEEC2ESt16initializer_listIiERKS0_


In [109]:
!objdump -t a.out | grep -i vector

0000000000400912  w    F .text	0000000000000026              _ZNSt12_Vector_baseIiSaIiEE11_M_allocateEm
0000000000400818  w    F .text	0000000000000079              _ZNSt6vectorIiSaIiEEC2ERKS1_
0000000000400938 g     F .text	0000000000000046              _Z3sumRKSt6vectorIiSaIiEE
0000000000400748 g     F .text	000000000000004a              _Z3mulRKSt6vectorIiSaIiEE
0000000000400892  w    F .text	000000000000007f              _ZNSt6vectorIiSaIiEEC2ESt16initializer_listIiERKS0_


Подчистим за собой:

In [ ]:
!rm -f *.o

<br />

##### External templates

Возможность сэкономить на времени компиляции, указав компилятору, что "где-то в другом месте этот шаблон уже скомпилирован, компилировать не надо, просто сделай внешнюю ссылку".

Подробно разбирать не будем из-за редкости использования.

https://stackoverflow.com/questions/8130602/using-extern-template-c11

<br />

##### Как определить константу в хедере

Объяснить варианты и разницу:

* `static`
    ```c++
    // in header / single cpp
    static const std::string s = "123";
    ```
* `inline`
    ```c++
    // in header / single cpp
    inline const std::string s = "123";
    ```
* `extern`
    ```c++
    // in header
    extern const std::string;
    
    // in a single cpp
    const std::string s = "123";
    ```
* `constexpr`
    ```c++
    // in header / single cpp
    constexpr std::array<const char*, 3> names = {"Dobrynia", "Alesha", "Ilya"};
    ```
    https://en.cppreference.com/w/cpp/language/constexpr Замечание: `constexpr` implies `inline`

<br />

##### Линковка, Link-Time-Optimization

Задача линковщика - собрать объектные файлы (.o/.obj) в исполняемый файл.

Линковщик должен:
1. Разрешить зависимости и построить связи
2. Смёржить символы, которым разрешено нарушать ODR

In [119]:
# compilation
!clang++ -Os -c ex5_templates/main.cpp
!clang++ -Os -c ex5_templates/util.cpp

# linking
!clang++ -Os main.o util.o -o a.out

# cleanup
!rm -f *.o a.out

https://docs.microsoft.com/en-us/cpp/build/reference/linking?view=vs-2019

https://gcc.gnu.org/onlinedocs/gcc/Link-Options.html

<br />

Пример LTO (разобрать чуть подробнее):

https://llvm.org/docs/LinkTimeOptimization.html#example-of-link-time-optimization

https://clang.llvm.org/docs/ThinLTO.html

<br />

##### Загрузка и запуск программы

https://en.cppreference.com/w/cpp/language/main_function

Что происходит, когда вы вызываете из командной строки `my_program.exe`? Или когда щёлкаете мышкой по exe-файлу в проводнике?

Простой вариант:
1. Загрузка кода программы с диска в ОП (возможны варианты)
2. Инициализация глобальных переменных и констант
    * в рамках одной единицы трансляции порядок гарантирован
    * между единицами трансляции порядок не определён
3. Выполнение `main`

Здесь баг:

```c++
// --- f1.h ---
#pragma once
extern const std::string s1;
        
// --- f1.cpp ---
#include "f1.h"
const std::string s1 = "123";
        
// --- f2.h ---
#pragma once
extern const std::string s2;
        
// --- f2.cpp ---
#include "f2.h"
#include "f1.h"
const std::string s2 = s1 + "456";

// --- main.cpp ---
#include "f2.h"

int main() {
    std::cout << s2 << std::endl;  // ???
    return 0;
}
```

__Вопрос__: Почему? Как это можно починить?


<details>
<summary>Вариант:</summary>
<p>

```c++
// --- f.h ---
#pragma once
extern const std::string s1;
extern const std::string s2;
        
// --- f.cpp ---
#include "f.h"
const std::string s1 = "123";
const std::string s2 = s1 + "456";

// --- main.cpp ---
#include "f.h"

int main() {
    std::cout << s2 << std::endl;
    return 0;
}
```

</p>
</details>

<br />

##### Запуск программы и shared libs

Код может линковаться не только со статическими билиотеками, но и с динамическими. Динамическая билиотека - набор функций, которые не вкопилируются в исполняемый файл, а хранятся отдельным файлом. В исполняемом файле прописывается специальная информация для функций, лежащих в динамических билиотеках.

Плюсы:
* Можно сэкономить место на диске и использовать одну динамическую библиотеку (например, libc) для нескольких программ
* Аналогичным образом экономится место в ОП под код
* Можно внести исправления в динамическую библиотеку, перекомпилировать саму программу при этом не нужно

Минусы:
* Нужно размечать экспортируемые функции вручную
* Невозможен ряд оптимизаций (inlining(!!!), LTO, constant propagation etc ...)
* Невозможен анализ компилятором неиспользуемого кода в динамической библиотеке (если функция помечена на экспорт, выкидывать её нельзя, и она тянет за собой всё по графу зависимостей)
* Нужно загружать динамическую библиотеку в память:
    * либо лениво при первом вызове функции
    * либо сразу при старте приложения (настраиваемо)
* Нужно проводить доп. работу по инициализации адресов функций
* Дороже вызов импортированной функции (особенно первый)

<br />